#Importing all the required Libraries.

In [1]:
pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.0 MB/s eta 0:00:00


In [2]:
import  pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

import pickle

from spellchecker import SpellChecker
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


#EDA on the Dataset

In [3]:
# Loading of the dataset
# Assuming your dataset is in a CSV file named 'your_dataset.csv'
df = pd.read_csv('Training.csv')

# Display basic information about the dataset
print(df.info())

# Display summary statistics
print(df.describe())

# Check for missing values
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Columns: 133 entries, itching to prognosis
dtypes: int64(132), object(1)
memory usage: 5.0+ MB
None
           itching    skin_rash  nodal_skin_eruptions  continuous_sneezing  \
count  4920.000000  4920.000000           4920.000000          4920.000000   
mean      0.137805     0.159756              0.021951             0.045122   
std       0.344730     0.366417              0.146539             0.207593   
min       0.000000     0.000000              0.000000             0.000000   
25%       0.000000     0.000000              0.000000             0.000000   
50%       0.000000     0.000000              0.000000             0.000000   
75%       0.000000     0.000000              0.000000             0.000000   
max       1.000000     1.000000              1.000000             1.000000   

         shivering       chills   joint_pain  stomach_pain      acidity  \
count  4920.000000  4920.000000  4920.000000   4

In [ ]:
# Visualize the distribution of the target variable with rotated x-axis labels and adjusted figure size
plt.figure(figsize=(12, 6))  # Adjust the figure size as needed
sns.countplot(x='prognosis', data=df)
plt.title('Distribution of Target Variable')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout for better spacing
plt.show()


In [ ]:
# Univariate analysis for some features
# Assuming 'itching', 'skin_rash', 'joint_pain' are three symptom columns
sns.countplot(x='itching', data=df)
plt.title('Distribution of Itching')
plt.show()


In [ ]:
sns.countplot(x='skin_rash', data=df)
plt.title('Distribution of Skin Rash')
plt.show()

In [ ]:
sns.countplot(x='joint_pain', data=df)
plt.title('Distribution of Joint Pain')
plt.show()

In [ ]:
# Correlation matrix and heatmap
plt.figure(figsize=(10, 10))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Box plot for a categorical feature (assuming there is a symptom column named 'itching')
plt.figure(figsize=(10, 10))
sns.countplot(x='prognosis', hue='itching', data=df)
plt.title('Distribution of Itching for Different Prognoses')
plt.legend(title='Itching', labels=['No', 'Yes'])
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout for better spacing
plt.show()


## Splitting the data into train and test, Label Encoding the "prognosis" column

In [4]:
df

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,...,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,...,0,0,1,1,1,1,0,0,0,Psoriasis


<H1> changes done here

In [5]:
# diseases_list = {
#     1: 'Fungal infection', 2: 'Allergy', 3: 'GERD', 4: 'Chronic cholestasis', 5: 'Drug Reaction', 6: 'Peptic ulcer disease',
#     7: 'AIDS', 8: 'Diabetes', 9: 'Gastroenteritis', 10: 'Bronchial Asthma', 11: 'Hypertension', 12: 'Migraine', 13: 'Cervical spondylosis',
#     14: 'Paralysis (brain hemorrhage)', 15: 'Jaundice', 16: 'Malaria', 17: 'Chicken pox', 18: 'Dengue', 19: 'Typhoid', 20: 'hepatitis A',
#     21: 'hepatitis B', 22: 'hepatitis C', 23: 'hepatitis D', 24: 'Hepatitis E', 25: 'Alcoholic hepatitis', 26: 'Tuberculosis',
#     27: 'Common Cold', 28: 'Pneumonia', 29: 'Dimorphic hemmorhoids(piles)', 30: 'Heart attack', 31: 'Varicose veins', 32: 'Hypothyroidism',
#     33: 'Hyperthyroidism', 34: 'Hypoglycemia', 35: 'Osteoarthristis', 36: 'Arthritis', 37: '(vertigo) Paroymsal  Positional Vertigo',
#     38: 'Acne', 39: 'Urinary tract infection', 40: 'Psoriasis', 41: 'Impetigo'
# }

diseases_list = {
    1: 'Fungal infection', 2: 'Allergy', 5: 'Drug Reaction', 8: 'Diabetes', 9: 'Gastroenteritis', 10: 'Bronchial Asthma', 11: 'Hypertension', 12: 'Migraine',
     15: 'Jaundice', 16: 'Malaria', 17: 'Chicken pox', 18: 'Dengue', 19: 'Typhoid', 20: 'hepatitis A',
    21: 'hepatitis B', 22: 'hepatitis C', 23: 'hepatitis D', 24: 'Hepatitis E', 25: 'Alcoholic hepatitis', 26: 'Tuberculosis',
    27: 'Common Cold', 28: 'Pneumonia', 29: 'Dimorphic hemmorhoids(piles)', 30: 'Heart attack', 31: 'Varicose veins', 32: 'Hypothyroidism',
     36: 'Arthritis',
    38: 'Acne'
}

reverse_diseases_list = {v: k for k, v in diseases_list.items()}

In [6]:
reverse_diseases_list = {k.lower(): v for k, v in reverse_diseases_list.items()}


In [ ]:
test_value = df['prognosis'].iloc[0]  # Example: get the first entry
mapped_value = reverse_diseases_list.get(test_value.lower(), test_value)
print(f"Original: {test_value}, Mapped: {mapped_value}")

Original: Fungal infection, Mapped: 1


In [7]:
df['prognosis'] = df['prognosis'].apply(lambda x: reverse_diseases_list.get(x.lower(), np.nan))




In [8]:
df_without_nan_columns = df.dropna()
df_shuffled = shuffle(df_without_nan_columns, random_state=20)


X = df_shuffled.drop('prognosis', axis=1)
y = df_shuffled['prognosis']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)

In [9]:
nan_count_prognosis = df['prognosis'].isna().sum()
print(f"Number of NaN values in 'prognosis': {nan_count_prognosis}")

Number of NaN values in 'prognosis': 1800


## Applying Various Models on the Dataset

In [10]:
model_name1 = "SVC"
model1 = SVC(kernel='linear')

model1.fit(X_train, y_train)

# Test the model
predictions = model1.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"{model_name1} Accuracy: {accuracy}")

# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)
print(f"{model_name1} Confusion Matrix:")
print(np.array2string(cm, separator=', '))

SVC Accuracy: 1.0
SVC Confusion Matrix:
[[36,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0, 40,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  0, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  0,  0, 28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  0,  0,  0, 36,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  0,  0,  0,  0, 37,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0, 

In [11]:

model_name2 = "Random Forest"
model2 = RandomForestClassifier(n_estimators=100, random_state=42)

model2.fit(X_train, y_train)

# Test the model
predictions = model2.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"{model_name2} Accuracy: {accuracy}")

# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)
print(f"{model_name2} Confusion Matrix:")
print(np.array2string(cm, separator=', '))

Random Forest Accuracy: 1.0
Random Forest Confusion Matrix:
[[36,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0, 40,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  0, 26,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  0,  0, 28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  0,  0,  0, 36,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
   0,  0,  0,  0,  0,  0,  0,  0],
 [ 0,  0,  0,  0,  0,  0,  0,  0, 37,  0,  0,  0,  0,  0,  0,  0,  0

In [ ]:
model_name3 = "Gradient Boosting"
model3 = GradientBoostingClassifier(n_estimators=100, random_state=42)

model3.fit(X_train, y_train)

# Test the model
predictions = model3.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"{model_name3} Accuracy: {accuracy}")

# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)
print(f"{model_name3} Confusion Matrix:")
print(np.array2string(cm, separator=', '))

Gradient Boosting Accuracy: 1.0
Gradient Boosting Confusion Matrix:
[[40,  0,  0, ...,  0,  0,  0],
 [ 0, 43,  0, ...,  0,  0,  0],
 [ 0,  0, 28, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 34,  0,  0],
 [ 0,  0,  0, ...,  0, 41,  0],
 [ 0,  0,  0, ...,  0,  0, 31]]


In [ ]:
model_name4 = "KNeighbors"
model4 = KNeighborsClassifier(n_neighbors=5)

model4.fit(X_train, y_train)

# Test the model
predictions = model4.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"{model_name4} Accuracy: {accuracy}")

# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)
print(f"{model_name4} Confusion Matrix:")
print(np.array2string(cm, separator=', '))

KNeighbors Accuracy: 1.0
KNeighbors Confusion Matrix:
[[40,  0,  0, ...,  0,  0,  0],
 [ 0, 43,  0, ...,  0,  0,  0],
 [ 0,  0, 28, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 34,  0,  0],
 [ 0,  0,  0, ...,  0, 41,  0],
 [ 0,  0,  0, ...,  0,  0, 31]]


In [ ]:
model_name5 = "Multinomial NB"
model5 = MultinomialNB()

model5.fit(X_train, y_train)

# Test the model
predictions = model5.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"{model_name5} Accuracy: {accuracy}")

# Calculate confusion matrix
cm = confusion_matrix(y_test, predictions)
print(f"{model_name5} Confusion Matrix:")
print(np.array2string(cm, separator=', '))

Multinomial NB Accuracy: 1.0
Multinomial NB Confusion Matrix:
[[40,  0,  0, ...,  0,  0,  0],
 [ 0, 43,  0, ...,  0,  0,  0],
 [ 0,  0, 28, ...,  0,  0,  0],
 ...,
 [ 0,  0,  0, ..., 34,  0,  0],
 [ 0,  0,  0, ...,  0, 41,  0],
 [ 0,  0,  0, ...,  0,  0, 31]]


## Dumping one of the model into a pickle file

In [ ]:
pickle.dump(model1,open('svc.pkl','wb'))

In [12]:
pickle.dump(model2,open('random.pkl','wb'))

In [ ]:
svc = pickle.load(open('svc.pkl','rb'))

In [13]:
randomForest = pickle.load(open('random.pkl','rb'))

In [14]:
sym_des = pd.read_csv("symtoms_df.csv")
precautions = pd.read_csv("precautions_df.csv")
workout = pd.read_csv("workout_df.csv")
description = pd.read_csv("description.csv")
medications = pd.read_csv('medications.csv')
diets = pd.read_csv("diets.csv")

In [15]:
symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis', 14: 'Drug Reaction', 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 30: 'Migraine', 7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice', 29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia', 13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 31: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}


# Function to preprocess and correct spelling of the text
def preprocess_and_correct(text, spell):
    # Lowercase, remove punctuation, split into words, and correct spelling
    corrected_words = []
    for word in text.lower().replace('_', ' ').split():
        corrected_word = spell.correction(word)
        # Only append if corrected_word is not None
        if corrected_word is not None:
            corrected_words.append(corrected_word)
        else:
            # If the corrected word is None, append the original word
            corrected_words.append(word)
    return ' '.join(corrected_words)

# Your symptoms dictionary

# Instantiate spell checker
spell = SpellChecker()

# # Create a corpus of unique symptoms from your dictionary, preprocessed and spell-corrected
symptom_corpus = [preprocess_and_correct(symptom, spell) for symptom in symptoms_dict.keys()]
vectorizer = TfidfVectorizer().fit(symptom_corpus)  # Train a TF-IDF vectorizer on the corrected symptoms corpus

# Function to find the closest symptom from the dictionary to the user's input
def find_closest_symptoms(user_symptoms, symptoms_dict, vectorizer, spell):
    corrected_user_symptoms = [preprocess_and_correct(symptom, spell) for symptom in user_symptoms]
    user_symptom_vectors = vectorizer.transform(corrected_user_symptoms)
    dict_symptom_vectors = vectorizer.transform(symptom_corpus)
    closest_symptoms = []
    for user_vector in user_symptom_vectors:
        similarities = cosine_similarity(user_vector, dict_symptom_vectors)
        closest_symptom_index = np.argmax(similarities)
        closest_symptom = list(symptoms_dict.keys())[closest_symptom_index]
        closest_symptoms.append(closest_symptom)
    return closest_symptoms

In [16]:
#============================================================
# custome and helping functions
#==========================helper funtions================
def helper(dis):
    desc = description[description['Disease'] == predicted_disease]['Description']
    desc = " ".join([w for w in desc])

    pre = precautions[precautions['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]
    pre = [col for col in pre.values]

    med = medications[medications['Disease'] == dis]['Medication']
    med = [med for med in med.values]

    die = diets[diets['Disease'] == dis]['Diet']
    die = [die for die in die.values]

    wrkout = workout[workout['disease'] == dis] ['workout']


    return desc,pre,med,die,wrkout

symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}

# Model Prediction function
def get_predicted_value(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    for item in patient_symptoms:
        input_vector[symptoms_dict[item]] = 1
    return diseases_list[randomForest.predict([input_vector])[0]]




In [23]:
# Test 1
# Split the user's input into a list of symptoms (assuming they are comma-separated) # itching,skin_rash,nodal_skin_eruptions
symptoms = input("Enter your symptoms.......")
user_symptoms = [s.strip() for s in symptoms.split(',')]
# Remove any extra characters, if any
user_symptoms = [symptom.strip("[]' ") for symptom in user_symptoms]
# closest_symptoms = find_closest_symptoms(user_symptoms, symptoms_dict, vectorizer, spell)
# print(closest_symptoms)
predicted_disease = get_predicted_value(user_symptoms)

desc, pre, med, die, wrkout = helper(predicted_disease)

print("=================predicted disease============")
print(predicted_disease)
print("=================description==================")
print(desc)
print("=================precautions==================")
i = 1
for p_i in pre[0]:
    print(i, ": ", p_i)
    i += 1

print("=================medications==================")
for m_i in med:
    print(i, ": ", m_i)
    i += 1

print("=================workout==================")
for w_i in wrkout:
    print(i, ": ", w_i)
    i += 1

print("=================diets==================")
for d_i in die:
    print(i, ": ", d_i)
    i += 1


Enter your symptoms.......runny_nose,redness_of_eyes,throat_irritation,continuous_sneezing,chest_pain
=================predicted disease============
Migraine
=================description==================
Migraine is a type of headache that often involves severe pain and sensitivity to light and sound.
=================precautions==================
1 :  meditation
2 :  reduce stress
3 :  use poloroid glasses in sun
4 :  consult doctor
=================medications==================
5 :  ['Analgesics', 'Triptans', 'Ergotamine derivatives', 'Preventive medications', 'Biofeedback']
=================workout==================
6 :  Identify and avoid trigger foods
7 :  Stay hydrated
8 :  Include magnesium-rich foods
9 :  Consume omega-3 fatty acids
10 :  Limit caffeine and alcohol
11 :  Consume riboflavin-rich foods
12 :  Limit processed foods
13 :  Maintain regular meal times
14 :  Consult a healthcare professional
15 :  Manage stress
=================diets==================
16 :  ['Migraine

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
